<a href="https://colab.research.google.com/github/punkmic/pyspark-concepts/blob/master/SQL_Options_in_Spark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/googleplaystore.csv.zip

Archive:  /content/googleplaystore.csv.zip
  inflating: googleplaystore.csv     
  inflating: __MACOSX/._googleplaystore.csv  


In [5]:
path = '/content/googleplaystore.csv'

# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [2]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('options').getOrCreate()

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [6]:
googlep = spark.read.csv(path, header=True, inferSchema=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [7]:
googlep.limit(3).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


As well as the schema to make sure all the column types were correctly infered

In [8]:
googlep.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [9]:
from pyspark.sql.types import IntegerType, FloatType
newdf = googlep.withColumn('Rating', googlep['Rating'].cast(FloatType())) \
               .withColumn('Reviews', googlep['Reviews'].cast(IntegerType())) \
               .withColumn('Price', googlep['Price'].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [11]:
googlep.createOrReplaceTempView('tempview')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [12]:
spark.sql('SELECT * FROM tempview WHERE Rating > 4.1').limit(3).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [14]:
sql_results = spark.sql("SELECT App, Rating FROM tempview WHERE Category = 'COMICS' AND Rating > 4.5")
sql_results.limit(3).toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [21]:
spark.sql("SELECT Category, SUM(Reviews) AS total_reviews FROM tempview GROUP BY Category ORDER BY 2 DESC").limit(1).toPandas()

,Category,total_reviews
0,GAME,1.585422e+09


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [28]:
spark.sql("SELECT App, Reviews FROM tempview ORDER BY Reviews DESC").show(1)

+--------------------+-------------+
|                 App|      Reviews|
+--------------------+-------------+
|"Women""s Health ...|weight lose)"|
+--------------------+-------------+
only showing top 1 row



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [32]:
spark.sql("SELECT * FROM tempview WHERE App LIKE '%dating%' ").limit(3).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"Meet, chat & date. Free dating app - Chocolate...",DATING,3.9,8661,9.5M,"1,000,000+",Free,0,Mature 17+,Dating,"April 3, 2018",0.1.11,4.0 and up
1,Friend Find: free chat + flirt dating app,DATING,NaN,23,11M,100+,Free,0,Mature 17+,Dating,"July 31, 2018",1.0,4.4 and up
2,Spine- The dating app,DATING,5.0,5,9.3M,500+,Free,0,Teen,Dating,"July 14, 2018",4.0,4.0.3 and up


## 6. Use SQL Transformer to display how many free apps there are in this list

In [34]:
from pyspark.ml.feature import SQLTransformer

In [35]:
sqlTrans = SQLTransformer(statement="SELECT COUNT(*) FROM __THIS__ WHERE Type = 'Free' ")
sqlTrans.transform(newdf).show()

+--------+
|count(1)|
+--------+
|   10037|
+--------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [38]:
sqlTrans = SQLTransformer(statement="SELECT Genres, COUNT(*) AS total FROM __THIS__ GROUP BY 1 ORDER BY 2 DESC")
sqlTrans.transform(newdf).show(5)

+-------------+-----+
|       Genres|total|
+-------------+-----+
|        Tools|  842|
|Entertainment|  623|
|    Education|  549|
|      Medical|  463|
|     Business|  460|
+-------------+-----+
only showing top 5 rows



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [39]:
sqlTrans = SQLTransformer(statement="SELECT App FROM __THIS__ WHERE Genres = 'Tools' AND Reviews > 100")
sqlTrans.transform(newdf).show(3)

+-----------------+
|              App|
+-----------------+
|Moto File Manager|
|           Google|
| Google Translate|
+-----------------+
only showing top 3 rows



## That's all folks! Great job!